<a href="https://colab.research.google.com/github/paulanavarretec/WineRec/blob/master/wineRec_OneHotEncoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello World!")

Hello World!
